In [1]:
import pandas as pd
import numpy as np
from hops import hdfs

path=hdfs.project_path() + "Jupyter/fsbook/data/ecommerce/"
#path = "../data/ecommerce/"

orders_df = pd.read_csv(path + "olist_orders_dataset.csv")
orders_df.head()

/srv/hops/anaconda/envs/theenv/lib/python3.8/site-packages/fsspec/implementations/hdfs.py:83: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  self.client = HadoopFileSystem(


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [2]:
df = orders_df.dropna()

# Only keep  delivered orders
df = df[df['order_status'] == 'delivered']

# Drop rows where order_purchase_timestamp <= order_approved_at <= order_delivered_carrier_date <= order_delivered_customer_date
df = df[~((df['order_purchase_timestamp'] >= df['order_approved_at']) | (df['order_approved_at'] >= df['order_delivered_carrier_date']) | (df['order_delivered_carrier_date'] >= df['order_delivered_customer_date']))]


def bucketize_actual_delivery_vs_expectation (row):
  if row['days_between_delivery_expectation'] <= 0 :
    return -1
  elif row['days_between_delivery_expectation'] <= 7 :
    return 1
  elif row['days_between_delivery_expectation'] <= 14 :
    return 2
  else:
    return 3


# Days between purchase and delivery dates
df['days_between_purhcase_and_delivery'] = (pd.to_datetime(df['order_delivered_customer_date']) - pd.to_datetime(df['order_purchase_timestamp'])).dt.days

# if the order was approved late or on time (0=on time, 1=late)
df['order_approved_late']=np.where((pd.to_datetime(df['order_approved_at']) - pd.to_datetime(df['order_purchase_timestamp'])).dt.days == 0, 0, 1)

# Actual delivery vs. Expected delivery: 1=Delivered before expected date, 2= Delivered one week later than expected date, 3= Delivered two weeks later than expected date, 4= Delivered more than two weeks later than expected date
df['days_between_delivery_expectation']=(pd.to_datetime(df['order_estimated_delivery_date']) - pd.to_datetime(df['order_delivered_customer_date'])).dt.days
df['actual_delivery_vs_expectation_bucket'] = df.apply (lambda row: bucketize_actual_delivery_vs_expectation(row), axis=1)

df = df[['order_id','days_between_purhcase_and_delivery','order_approved_late','actual_delivery_vs_expectation_bucket','order_delivered_carrier_date']]

orders_df = df 
orders_df

,order_id,days_between_purhcase_and_delivery,order_approved_late,actual_delivery_vs_expectation_bucket,order_delivered_carrier_date
0,e481f51cbdc54678b7cc49136f2d6af7,8,0,1,2017-10-04 19:55:00
1,53cdb2fc8bc7dce0b6741e2150273451,13,1,1,2018-07-26 14:31:00
2,47770eb9100c2d0c44946d9cf07ec65d,9,0,3,2018-08-08 13:50:00
3,949d5b44dbf5de918fe9c16f97b45f8a,13,0,2,2017-11-22 13:39:59
4,ad21c59c0840e6cb83a9ceb5573f8159,2,0,2,2018-02-14 19:46:34
...,...,...,...,...,...
99435,880675dff2150932f1601e1c07eadeeb,11,0,3,2017-03-01 10:22:52
99437,63943bddc261676b46f01ca7ac2f7bd8,22,0,1,2018-02-07 23:22:42
99438,83c1379a015df1e13d02aae0204711ab,24,0,1,2017-08-28 20:52:26
99439,11c177c8e97725db2631073c19f07b62,17,0,3,2018-01-12 15:35:03


In [6]:
items_df = pd.read_csv(path + "olist_order_items_dataset.csv")

items_df = items_df[['order_id','shipping_limit_date','price','freight_value']].dropna()

# Price and Freight Value must be non-negative
df = items_df[(items_df['price']>=0) & (items_df['freight_value']>=0)]

df1 = df.groupby('order_id').agg(total_order_price=('price', 'sum'), 
                                 total_order_freight=('freight_value','sum'),
                                 max_shipping_limit_date=('shipping_limit_date','max')).reset_index()

# Extract 1 feature: is_multiItems_order -- If the order has multiple items or not (1 or 0)
df2 = df.groupby('order_id').agg(cnt=('price', 'count')).reset_index()
df2['is_multiItems_order'] = np.where(df2['cnt'] > 1, 1, 0)

items_df = df1.merge(df2, how="inner", on='order_id')[['order_id','is_multiItems_order','total_order_price','total_order_freight','max_shipping_limit_date']]

items_df

,order_id,is_multiItems_order,total_order_price,total_order_freight,max_shipping_limit_date
0,00010242fe8c5a6d1ba2dd792cb16214,0,58.90,13.29,2017-09-19 09:45:35
1,00018f77f2f0320c557190d7a144bdd3,0,239.90,19.93,2017-05-03 11:05:13
2,000229ec398224ef6ca0657da4fc703e,0,199.00,17.87,2018-01-18 14:48:30
3,00024acbcdf0a6daa1e931b038114c75,0,12.99,12.79,2018-08-15 10:10:18
4,00042b26cf59d7ce69dfabb4e55b4fd9,0,199.90,18.14,2017-02-13 13:57:51
...,...,...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,0,299.99,43.41,2018-05-02 04:11:01
98662,fffcd46ef2263f404302a634eb57f7eb,0,350.00,36.53,2018-07-20 04:31:48
98663,fffce4705a9662cd70adb13d4a31832d,0,99.90,16.95,2017-10-30 17:14:25
98664,fffe18544ffabc95dfada21779c9644f,0,55.99,8.72,2017-08-21 00:04:32


In [3]:
# Load the reviews table from csv file
reviews_df = pd.read_csv(path + "olist_order_reviews_dataset.csv")

# Drop extra reviews if an order has multiple order review scores
df_reviews = reviews_df.groupby('order_id', as_index= False).agg(review_score=('review_score', 'max'))
df_reviews

,order_id,review_score
0,00010242fe8c5a6d1ba2dd792cb16214,5
1,00018f77f2f0320c557190d7a144bdd3,4
2,000229ec398224ef6ca0657da4fc703e,5
3,00024acbcdf0a6daa1e931b038114c75,4
4,00042b26cf59d7ce69dfabb4e55b4fd9,5
...,...,...
99436,fffc94f6ce00a00581880bf54a75a037,5
99437,fffcd46ef2263f404302a634eb57f7eb,5
99438,fffce4705a9662cd70adb13d4a31832d,5
99439,fffe18544ffabc95dfada21779c9644f,5


In [7]:
orders_data = orders_df
items_data = items_df
reviews_data = df_reviews

In [9]:
!pip3 uninstall hsfs -y
!pip3 install 'git+https://github.com/logicalclocks/feature-store-api@master#egg=hsfs[python]&subdirectory=python'
#!pip3 install 'git+https://github.com/moritzmeister/feature-store-api@py39#egg=hsfs[python]&subdirectory=python'

  Cloning https://github.com/logicalclocks/feature-store-api (to revision master) to /tmp/pip-install-x8n8kl0u/hsfs_917e66c1a42046aeb63a005409304bd8
  Running command git clone -q https://github.com/logicalclocks/feature-store-api /tmp/pip-install-x8n8kl0u/hsfs_917e66c1a42046aeb63a005409304bd8
  Resolved https://github.com/logicalclocks/feature-store-api to commit cfa35380df447389aabf931067d46c40c0db44df
  Created wheel for hsfs: filename=hsfs-2.6.0.dev1-py3-none-any.whl size=168446 sha256=50ea369fc8177560ab8ad10db91ea751cf26b153ccbd46b3a4838b3a922edfb4
  Stored in directory: /tmp/pip-ephem-wheel-cache-a5lc2v2t/wheels/89/7c/26/95cbc541a5ee439a6bd9a4ca2d95e93e6978913447f4b2269d
Successfully built hsfs


In [10]:
import sys
print(sys.version)

3.8.11 (default, Aug  3 2021, 15:09:35) 
[GCC 7.5.0]


In [11]:
!pip show pyspark

In [1]:
import hsfs

import hsfs
connection = hsfs.connection()
fs = connection.get_feature_store()

# connection = hsfs.connection(
#     host="35.195.111.34",
#     project="rec",
#     secrets_store="local",
#     api_key_file="./api-key.txt",
#     engine="python"
# )
# fs = connection.get_feature_store()



Connected. Call `.close()` to terminate connection gracefully.


In [24]:
orders_data = orders_df
items_data = items_df
reviews_data = df_reviews

orders_fg = fs.create_feature_group(
                        name="orders",
                        primary_key=["order_id"],
                        version=1,
                        description="Order details",
                        online_enabled=True
)

orders_fg.save(orders_data)

Feature Group created successfully, explore it at https://fvhead.europe-west1-b.c.hops-20.internal/p/119/fs/67/fg/24
Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://fvhead.europe-west1-b.c.hops-20.internal/p/119/jobs/named/orders_1_offline_fg_backfill/executions


In [25]:
items_fg = fs.create_feature_group(
                        name="items",
                        primary_key=["order_id"],
                        version=1,
                        description="Order Item details",
                        online_enabled=True
)

items_fg.save(items_data)

Feature Group created successfully, explore it at https://fvhead.europe-west1-b.c.hops-20.internal/p/119/fs/67/fg/25
Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://fvhead.europe-west1-b.c.hops-20.internal/p/119/jobs/named/items_1_offline_fg_backfill/executions


In [28]:
reviews_fg = fs.create_feature_group(
                        name="reviews",
                        primary_key=["order_id"],
                        version=1,
                        description="Review details",
                        online_enabled=True
)

reviews_fg.save(reviews_data)

Feature Group created successfully, explore it at https://fvhead.europe-west1-b.c.hops-20.internal/p/119/fs/67/fg/26
Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://fvhead.europe-west1-b.c.hops-20.internal/p/119/jobs/named/reviews_1_offline_fg_backfill/executions


In [16]:
query = orders_fg.select(['order_id','order_delivered_carrier_date']).join(items_fg.select(['max_shipping_limit_date']))
derived_df = query.read()

2022-06-01 06:36:27,131 INFO: USE `demo_fs_meb10000_featurestore`
2022-06-01 06:36:27,856 INFO: SELECT `fg1`.`order_id` `order_id`, `fg1`.`order_delivered_carrier_date` `order_delivered_carrier_date`, `fg0`.`max_shipping_limit_date` `max_shipping_limit_date`
FROM `demo_fs_meb10000_featurestore`.`orders_3` `fg1`
INNER JOIN `demo_fs_meb10000_featurestore`.`items_2` `fg0` ON `fg1`.`order_id` = `fg0`.`order_id`


In [17]:
derived_df['days_between_delivered_carrier_and_shipping_limit'] = (pd.to_datetime(derived_df['order_delivered_carrier_date']) - pd.to_datetime(derived_df['max_shipping_limit_date'])).dt.days
derived_df

,order_id,order_delivered_carrier_date,max_shipping_limit_date,days_between_delivered_carrier_and_shipping_limit
0,92f53cdbef0f16b315c62faa337c4672,2018-07-25 14:21:00,2018-07-31 16:05:12,-7
1,1498b25563986950783a674b05e9a526,2018-01-26 17:59:15,2018-02-01 03:16:52,-6
2,b51f2621265ffd44eb7cdc9358836225,2017-11-28 14:33:02,2017-12-05 03:11:21,-7
3,16fd9bbafac5533abf2bd1b5bec61ec4,2017-04-18 16:09:53,2017-04-18 02:43:01,0
4,6559afcb08c9458feea561826c4a178d,2018-07-11 11:36:00,2018-07-11 15:10:14,-1
...,...,...,...,...
93825,c6f3dad60242a9cb7ce0486b495862bd,2018-02-23 10:47:18,2018-02-28 09:29:30,-5
93826,ad0d46c22c5a9dd2699f5142b1882f5f,2017-02-09 16:29:43,2017-02-12 17:55:05,-4
93827,756575f4894e97980e45d0e8710ee27d,2017-03-31 21:04:30,2017-04-03 20:45:13,-3
93828,2638e64a928557c96574eb38ef0b04e8,2018-05-03 11:08:00,2018-05-09 08:31:28,-6


In [18]:
derived_df['seller_shipped_late'] = \
    np.where(derived_df['days_between_delivered_carrier_and_shipping_limit'] > 0 , 1 ,0) 
derived_df

,order_id,order_delivered_carrier_date,max_shipping_limit_date,days_between_delivered_carrier_and_shipping_limit,seller_shipped_late
0,92f53cdbef0f16b315c62faa337c4672,2018-07-25 14:21:00,2018-07-31 16:05:12,-7,0
1,1498b25563986950783a674b05e9a526,2018-01-26 17:59:15,2018-02-01 03:16:52,-6,0
2,b51f2621265ffd44eb7cdc9358836225,2017-11-28 14:33:02,2017-12-05 03:11:21,-7,0
3,16fd9bbafac5533abf2bd1b5bec61ec4,2017-04-18 16:09:53,2017-04-18 02:43:01,0,0
4,6559afcb08c9458feea561826c4a178d,2018-07-11 11:36:00,2018-07-11 15:10:14,-1,0
...,...,...,...,...,...
93825,c6f3dad60242a9cb7ce0486b495862bd,2018-02-23 10:47:18,2018-02-28 09:29:30,-5,0
93826,ad0d46c22c5a9dd2699f5142b1882f5f,2017-02-09 16:29:43,2017-02-12 17:55:05,-4,0
93827,756575f4894e97980e45d0e8710ee27d,2017-03-31 21:04:30,2017-04-03 20:45:13,-3,0
93828,2638e64a928557c96574eb38ef0b04e8,2018-05-03 11:08:00,2018-05-09 08:31:28,-6,0


In [19]:
derived_df = derived_df.drop(columns=['order_delivered_carrier_date', \
                                  'max_shipping_limit_date','days_between_delivered_carrier_and_shipping_limit'])
derived_df

,order_id,seller_shipped_late
0,92f53cdbef0f16b315c62faa337c4672,0
1,1498b25563986950783a674b05e9a526,0
2,b51f2621265ffd44eb7cdc9358836225,0
3,16fd9bbafac5533abf2bd1b5bec61ec4,0
4,6559afcb08c9458feea561826c4a178d,0
...,...,...
93825,c6f3dad60242a9cb7ce0486b495862bd,0
93826,ad0d46c22c5a9dd2699f5142b1882f5f,0
93827,756575f4894e97980e45d0e8710ee27d,0
93828,2638e64a928557c96574eb38ef0b04e8,0


In [20]:
(derived_df['seller_shipped_late'] == 1).sum()

4294

In [21]:
(derived_df['seller_shipped_late'] == 0).sum()

89536

In [ ]:
derived_fg = fs.create_feature_group(
                        name="orders_late_shipped",
                        primary_key=["order_id"],
                        version=1,
                        description="Orders shippped late",
                        online_enabled=True
)

derived_fg.save(derived_df)

In [2]:
orders_fg = fs.get_feature_group(name="orders", version=1)
items_fg = fs.get_feature_group(name="items", version=1)
reviews_fg = fs.get_feature_group(name="reviews", version=1)
derived_fg = fs.get_feature_group(name="orders_late_shipped", version=1)

In [30]:
query2 = orders_fg.select_all().join(items_fg.select(
    ['is_multiItems_order', 'total_order_price', 'total_order_freight']))\
    .join(reviews_fg.select(['review_score'])).join(derived_fg.select(['seller_shipped_late']))
training_df = query2.read()
training_df

2022-06-01 08:14:01,586 INFO: USE `demo_fs_meb10000_featurestore`
2022-06-01 08:14:02,325 INFO: SELECT `fg3`.`order_id` `order_id`, `fg3`.`days_between_purhcase_and_delivery` `days_between_purhcase_and_delivery`, `fg3`.`order_approved_late` `order_approved_late`, `fg3`.`actual_delivery_vs_expectation_bucket` `actual_delivery_vs_expectation_bucket`, `fg3`.`order_delivered_carrier_date` `order_delivered_carrier_date`, `fg0`.`is_multiitems_order` `is_multiitems_order`, `fg0`.`total_order_price` `total_order_price`, `fg0`.`total_order_freight` `total_order_freight`, `fg1`.`review_score` `review_score`, `fg2`.`seller_shipped_late` `seller_shipped_late`
FROM `demo_fs_meb10000_featurestore`.`orders_1` `fg3`
INNER JOIN `demo_fs_meb10000_featurestore`.`items_1` `fg0` ON `fg3`.`order_id` = `fg0`.`order_id`
INNER JOIN `demo_fs_meb10000_featurestore`.`reviews_1` `fg1` ON `fg3`.`order_id` = `fg1`.`order_id`
INNER JOIN `demo_fs_meb10000_featurestore`.`orders_late_shipped_1` `fg2` ON `fg3`.`order_id`

,order_id,days_between_purhcase_and_delivery,order_approved_late,actual_delivery_vs_expectation_bucket,order_delivered_carrier_date,is_multiitems_order,total_order_price,total_order_freight,review_score,seller_shipped_late
0,00010242fe8c5a6d1ba2dd792cb16214,7,0,2,2017-09-19 18:34:16,0,58.90,13.29,5,0
1,00018f77f2f0320c557190d7a144bdd3,16,0,1,2017-05-04 14:35:00,0,239.90,19.93,4,1
2,000229ec398224ef6ca0657da4fc703e,7,0,2,2018-01-16 12:36:48,0,199.00,17.87,5,0
3,00024acbcdf0a6daa1e931b038114c75,6,0,1,2018-08-10 13:28:00,0,12.99,12.79,4,0
4,00042b26cf59d7ce69dfabb4e55b4fd9,25,0,3,2017-02-16 09:46:09,0,199.90,18.14,5,1
...,...,...,...,...,...,...,...,...,...,...
93825,fffc94f6ce00a00581880bf54a75a037,17,1,1,2018-04-25 12:09:00,0,299.99,43.41,5,0
93826,fffcd46ef2263f404302a634eb57f7eb,9,1,2,2018-07-17 08:05:00,0,350.00,36.53,5,0
93827,fffce4705a9662cd70adb13d4a31832d,4,1,2,2017-10-26 15:13:14,0,99.90,16.95,5,0
93828,fffe18544ffabc95dfada21779c9644f,1,0,2,2017-08-15 19:02:53,0,55.99,8.72,5,0


In [31]:
#trans_fn = hsfs.transformations.minmaxscalar


fv = fs.create_feature_view(name="order_reviews",
                            description="Dataset to train the order review model",
                            version = 1,
#s                            transformations= {"myfeature" : trans_fn},
                            label = ["review_score"],
                            query = query2)

Feature view created successfully, explore it at https://fvhead.europe-west1-b.c.hops-20.internal/p/119/fs/67/fv/order_reviews/version/2


In [33]:
train_ds = fv.create_training_dataset(
    version = 1,
#    start_time="2017-01-01 00:00",
    description = 'reviews dataset',
    data_format = 'csv',
    coalesce = True,
    splits = {'train': 80, 'test': 20},
#    train_split = "train",
    write_options = {'wait_for_job': True}
)

Training dataset job started successfully, you can follow the progress at https://fvhead.europe-west1-b.c.hops-20.internal/p/119/jobs/named/order_reviews_2_1_create_fv_td_01062022082042/executions


In [3]:
fv = fs.get_feature_view(name="order_reviews", version=2)

In [36]:
fvs = fs.get_feature_views("order_reviews")

In [16]:
!pip3 uninstall hsml -y
!pip3 install 'git+https://github.com/logicalclocks/machine-learning-api@main#egg=hsml&subdirectory=python'

  Cloning https://github.com/logicalclocks/machine-learning-api (to revision main) to /tmp/pip-install-fktr4rdy/hsml_a4c38f8fd45f4e4e8339719073c212f5
  Running command git clone -q https://github.com/logicalclocks/machine-learning-api /tmp/pip-install-fktr4rdy/hsml_a4c38f8fd45f4e4e8339719073c212f5
  Resolved https://github.com/logicalclocks/machine-learning-api to commit 478f493bbc3009939ce0a557329add9cbd923194
  Created wheel for hsml: filename=hsml-2.6.0.dev1-py3-none-any.whl size=89971 sha256=23d2ade52c96163c9c1b8368ef8037f5838bd97b565dacf9b87bed7ce77242bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-lcibcjbl/wheels/88/bc/8e/19063dd38b55ab47312e325972be1b7df2458a7926321af1f9
Successfully built hsml


In [4]:
td_version, df  = fv.get_training_dataset(version=2)
df

2022-06-01 09:25:19,989 INFO: USE `demo_fs_meb10000_featurestore`
2022-06-01 09:25:20,709 INFO: SELECT `fg3`.`order_id` `order_id`, `fg3`.`days_between_purhcase_and_delivery` `days_between_purhcase_and_delivery`, `fg3`.`order_approved_late` `order_approved_late`, `fg3`.`actual_delivery_vs_expectation_bucket` `actual_delivery_vs_expectation_bucket`, `fg3`.`order_delivered_carrier_date` `order_delivered_carrier_date`, `fg0`.`is_multiitems_order` `is_multiitems_order`, `fg0`.`total_order_price` `total_order_price`, `fg0`.`total_order_freight` `total_order_freight`, `fg1`.`review_score` `review_score`, `fg2`.`seller_shipped_late` `seller_shipped_late`
FROM `demo_fs_meb10000_featurestore`.`orders_1` `fg3`
INNER JOIN `demo_fs_meb10000_featurestore`.`items_1` `fg0` ON `fg3`.`order_id` = `fg0`.`order_id`
INNER JOIN `demo_fs_meb10000_featurestore`.`reviews_1` `fg1` ON `fg3`.`order_id` = `fg1`.`order_id`
INNER JOIN `demo_fs_meb10000_featurestore`.`orders_late_shipped_1` `fg2` ON `fg3`.`order_id`

,order_id,days_between_purhcase_and_delivery,order_approved_late,actual_delivery_vs_expectation_bucket,order_delivered_carrier_date,is_multiitems_order,total_order_price,total_order_freight,review_score,seller_shipped_late
0,00010242fe8c5a6d1ba2dd792cb16214,7,0,2,2017-09-19 18:34:16,0,58.90,13.29,5,0
1,00018f77f2f0320c557190d7a144bdd3,16,0,1,2017-05-04 14:35:00,0,239.90,19.93,4,1
2,000229ec398224ef6ca0657da4fc703e,7,0,2,2018-01-16 12:36:48,0,199.00,17.87,5,0
3,00024acbcdf0a6daa1e931b038114c75,6,0,1,2018-08-10 13:28:00,0,12.99,12.79,4,0
4,00042b26cf59d7ce69dfabb4e55b4fd9,25,0,3,2017-02-16 09:46:09,0,199.90,18.14,5,1
...,...,...,...,...,...,...,...,...,...,...
93825,fffc94f6ce00a00581880bf54a75a037,17,1,1,2018-04-25 12:09:00,0,299.99,43.41,5,0
93826,fffcd46ef2263f404302a634eb57f7eb,9,1,2,2018-07-17 08:05:00,0,350.00,36.53,5,0
93827,fffce4705a9662cd70adb13d4a31832d,4,1,2,2017-10-26 15:13:14,0,99.90,16.95,5,0
93828,fffe18544ffabc95dfada21779c9644f,1,0,2,2017-08-15 19:02:53,0,55.99,8.72,5,0


In [5]:
from sklearn.model_selection import train_test_split

def data_split(data):
  training_data, testing_data = train_test_split(data, test_size=0.1, random_state=42)
  X_train, X_valid, y_train, y_valid = train_test_split(training_data.drop(
      ['review_score', 'order_id', 'order_delivered_carrier_date'], axis=1), 
    training_data['review_score'], test_size=0.1, random_state=7)
  data_pair = [(X_train, y_train), (X_valid, y_valid)]
  return data_pair

df2 = data_split(df)

In [6]:
data_pair = df2 #data_split(training_data)

train_data_X = data_pair[0][0]
train_data_X

,days_between_purhcase_and_delivery,order_approved_late,actual_delivery_vs_expectation_bucket,is_multiitems_order,total_order_price,total_order_freight,seller_shipped_late
76378,11,0,3,0,35.00,16.05,0
73272,7,1,3,0,169.90,13.53,0
76656,2,0,2,0,139.90,23.56,0
7919,11,0,1,0,99.00,15.44,0
32862,15,0,1,0,120.00,34.20,0
...,...,...,...,...,...,...,...
47871,7,0,2,0,34.99,8.72,0
33729,8,0,3,1,181.90,27.98,0
7721,22,0,3,0,228.00,26.09,0
11857,18,0,2,0,259.99,16.57,0


In [7]:
train_data_Y = data_pair[0][1]
train_data_Y

76378    3
73272    5
76656    4
7919     4
32862    4
        ..
47871    5
33729    5
7721     3
11857    5
15212    4
Name: review_score, Length: 76002, dtype: int64

In [8]:
test_data_X = data_pair[1][0]
test_data_X

,days_between_purhcase_and_delivery,order_approved_late,actual_delivery_vs_expectation_bucket,is_multiitems_order,total_order_price,total_order_freight,seller_shipped_late
14495,17,0,-1,1,25.00,30.46,1
72178,7,0,2,0,35.00,8.88,0
83704,16,0,1,0,12.90,18.23,0
32530,20,1,1,0,24.99,14.10,0
77875,8,0,2,0,110.00,15.52,0
...,...,...,...,...,...,...,...
23243,8,1,2,0,42.00,12.23,0
83048,4,0,3,0,249.99,19.00,0
68255,31,0,1,0,39.00,18.23,0
66112,8,1,-1,0,179.90,23.01,0


In [9]:
test_data_Y = data_pair[1][1]
test_data_Y

14495    1
72178    4
83704    4
32530    5
77875    4
        ..
23243    4
83048    3
68255    3
66112    1
92907    5
Name: review_score, Length: 8445, dtype: int64

In [10]:
import xgboost
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import time
import hsml
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

def model_performance(X_train, y_train, eval_set):
  param_dict = {
    'colsample_bytree' : 1.0,
    'learning_rate': 0.02,
    'max_depth': 5,
    'min_child_weight': 10,
    'subsample' : 0.5
  }

  xgb_model = XGBRegressor(n_estimators=1000, objective='reg:squarederror', colsample_bytree = param_dict['colsample_bytree'], learning_rate=param_dict['learning_rate'], max_depth=param_dict['max_depth'], min_child_weight=param_dict['min_child_weight'], subsample=param_dict['subsample'])

  xgb_model.fit(X_train, y_train, eval_metric='rmse', eval_set=eval_set, verbose=False)
  return xgb_model.evals_result()

def test_model(xgb_model: XGBRegressor, test_data_X, test_data_Y):
  yhat = xgb_model.predict(test_data_X)

    
param_dict = {
'colsample_bytree' : 1.0,
'learning_rate': 0.02,
'max_depth': 5,
'min_child_weight': 10,
'subsample' : 0.5
}

#   training_data = df_merged
#training_data = fv.get_training_dataset(version=3)

#data_pair = data_split(training_data)

train_data_X = data_pair[0][0]
train_data_Y = data_pair[0][1]

test_data_X = data_pair[1][0]
test_data_Y = data_pair[1][1]

xgb_model_final = XGBRegressor(objective='reg:squarederror', n_estimators=200, colsample_bytree = param_dict['colsample_bytree'], learning_rate=param_dict['learning_rate'], max_depth=param_dict['max_depth'], min_child_weight=param_dict['min_child_weight'], subsample=param_dict['subsample'])
xgb_model_final.fit(train_data_X, train_data_Y,verbose=True)

metrics = model_performance(train_data_X,train_data_Y, data_pair)
test_model(xgb_model_final, test_data_X, test_data_Y)

#return xgb_model_final
metrics

2022-06-01 09:25:51,991 INFO: generated new fontManager


{'validation_0': {'rmse': [3.795406,
   3.726711,
   3.659544,
   3.593935,
   3.529863,
   3.467246,
   3.405844,
   3.345739,
   3.287057,
   3.229746,
   3.173839,
   3.11904,
   3.065574,
   3.013274,
   2.962023,
   2.912197,
   2.86329,
   2.815577,
   2.769077,
   2.723591,
   2.679208,
   2.63588,
   2.593411,
   2.55205,
   2.511742,
   2.472225,
   2.433868,
   2.396437,
   2.359916,
   2.324314,
   2.289531,
   2.255711,
   2.22266,
   2.190473,
   2.159094,
   2.128557,
   2.098845,
   2.069883,
   2.041661,
   2.014109,
   1.987376,
   1.961441,
   1.936225,
   1.911585,
   1.887725,
   1.864416,
   1.841757,
   1.819664,
   1.798222,
   1.777364,
   1.757174,
   1.737517,
   1.718426,
   1.699831,
   1.68193,
   1.664432,
   1.647394,
   1.630985,
   1.614905,
   1.599368,
   1.584264,
   1.5697,
   1.555557,
   1.541821,
   1.528626,
   1.515785,
   1.503356,
   1.49127,
   1.47965,
   1.468338,
   1.457355,
   1.446787,
   1.436481,
   1.426576,
   1.41704,
   1.407711,

In [15]:
import os
import joblib
MODEL_DIR="model"
REVIEW_MODEL_PKL = MODEL_DIR + "/review.pkl"

if os.path.isdir(MODEL_DIR) == False :
    os.mkdir(MODEL_DIR)
joblib.dump(xgb_model_final, REVIEW_MODEL_PKL)


['model/review.pkl']

In [19]:
import hsml
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(train_data_X)
output_schema = Schema(train_data_Y)

conn = hsml.connection()

# conn = hsml.connection(
#     host="35.195.111.34",
#     project="rec",
#     hostname_verification=True,
# #    api_key_file="./api-key.txt"
#     api_key_value="ooeY4kuI1WMhA8Xh.m0yNQw70N2lSn2zOLwL1ta5eZeDmHqJH4YUKRM8rANqAPGihMpjd8TRMt69GVAjb"
#     )
mr = conn.get_model_registry()

sk_model = mr.sklearn.create_model("review_score", 
                       metrics={ "accuracy" : 0.5},
                       input_example=train_data_X,
                       model_schema=ModelSchema(input_schema=input_schema, output_schema=output_schema))

sk_model.save(MODEL_DIR)

Connected. Call `.close()` to terminate connection gracefully.



Exported model review_score with version 1


Model(name: 'review_score', version: 1)

In [ ]:
test_sample = df_merged.drop(['review_score', 'order_id'], axis=1).sample()
predicted_review_score = model.predict(test_sample)
print("PREDICTED REVIEW SCORE [1-5]: ",predicted_review_score)
